[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:

with open("occupied.json", "rt") as INFILE:
    data=json.load(INFILE)
data

{'data': [{'date': '2023-11-24', 'occupied': None, 'hospital': 2264},
  {'date': '2023-11-17', 'occupied': None, 'hospital': 2503},
  {'date': '2023-11-10', 'occupied': None, 'hospital': 2703},
  {'date': '2023-11-03', 'occupied': None, 'hospital': 3236},
  {'date': '2023-10-27', 'occupied': None, 'hospital': 3622},
  {'date': '2023-10-20', 'occupied': None, 'hospital': 3779},
  {'date': '2023-10-13', 'occupied': None, 'hospital': 4377},
  {'date': '2023-10-06', 'occupied': None, 'hospital': 4312},
  {'date': '2023-09-29', 'occupied': None, 'hospital': 3675},
  {'date': '2023-09-22', 'occupied': None, 'hospital': 3051},
  {'date': '2023-09-15', 'occupied': None, 'hospital': 3017},
  {'date': '2023-09-08', 'occupied': None, 'hospital': 3188},
  {'date': '2023-09-01', 'occupied': None, 'hospital': 2791},
  {'date': '2023-08-25', 'occupied': None, 'hospital': 2543},
  {'date': '2023-08-18', 'occupied': None, 'hospital': 2495},
  {'date': '2023-08-11', 'occupied': None, 'hospital': 2232},


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [5]:

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    print (startdate, ' to ', enddate)
    index=pd.date_range(startdate, enddate, freq='7D')
    occupieddf=pd.DataFrame(index=index, columns=['occupied', 'hospital'])
    
    for entry in datalist:
        date = parse_date(entry['date'])
        for column in ['occupied', 'hospital']:
            if pd.isna(occupieddf.loc[date, column]): 
                value = float(entry[column]) if entry[column] is not None else 0.0
                occupieddf.loc[date, column] = value

    occupieddf.fillna(0.0, inplace=True)
    return occupieddf

df=wrangle_data(data) 
df



2020-03-20 00:00:00  to  2023-11-24 00:00:00


,occupied,hospital
2020-03-20,0.0,1580.0
2020-03-27,0.0,6462.0
2020-04-03,1788.0,13635.0
2020-04-10,2820.0,18669.0
2020-04-17,2780.0,16728.0
...,...,...
2023-10-27,0.0,3622.0
2023-11-03,0.0,3236.0
2023-11-10,0.0,2703.0
2023-11-17,0.0,2503.0


## 
Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [6]:

def access_api():
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    filters = ['areaType=nation', 'areaName=England']
    structure = {
    "date": "date",
    "occupied": "covidOccupiedMVBedsWeekly",
    "hospital": "hospitalCasesWeekly", 
}
    api = Cov19API(filters=filters, structure=structure)
    occupied=api.get_json()
    with open("occupied.json","wt") as OUTF:
        json.dump(occupied, OUTF)
    with open("occupied.json", "rt") as INFILE:
        data=json.load(INFILE)


    return data 

In [7]:

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
   
   
    
apibutton=wdg.Button(
    description='Update',
    disabled=False,
    button_style='danger',
    tooltip="Keep calm and carry on",
    icon='exclamation-triangle'
)

apibutton.on_click(api_button_callback) 

display(apibutton)



Button(button_style='danger', description='Update', icon='exclamation-triangle', style=ButtonStyle(), tooltip=…

2020-03-20 00:00:00  to  2023-11-24 00:00:00


## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [8]:
def plot_random_class(type):
    """ Our sample graph plotting function """
    df[list(type)].plot()
    plt.show() 


classnumber=wdg.SelectMultiple(
    options=['occupied', 'hospital'],
    value=['occupied'],
    row=2,
    description='type',
    disabled=False,
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=classnumber.value
    if current==classnumber.options[0]:
        other=classnumber.options[1]
    else:
        other=classnumber.options[0]
    classnumber.value=[other] 
    classnumber.value=current 
    
  
graph=wdg.interactive_output(plot_random_class, {'type': classnumber})

display(classnumber, graph)

# Description of the graph:In this chart I am comparing the data of occupied and hospital. 
#"Occupied" represents covidOccupiedMVBedsWeekly and "hospital" represents hospitalCasesWeekly. 
#The blue walk in this graph represents "occupied" and the yellow one represents "hospital".
#In the widget named type, if you select "occupied", the chart will show the walk of occupied, and if you 
#select "hospital", the chart will show the walk of hospital. If you select both occupied and hospital at the 
#same time, the chart will show the two walks of occupied and hospital,
#which makes it easier to compare the two data in the same chart.


SelectMultiple(description='type', index=(0,), options=('occupied', 'hospital'), value=('occupied',))

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).